#### imports and function defintions

In [1]:
from datetime import date
import numpy as np
import pandas as pd
from scipy.stats import norm
import cufflinks as cf

cf.go_offline()

N = norm.cdf

def d1(S,K,r,q,T,sigma):
    return (np.log(S/K)+(r-q+sigma**2/2)*T)/(sigma*np.sqrt(T))

def d2(S,K,r,q,T,sigma):
    return d1(S,K,r,q,T,sigma)-sigma*np.sqrt(T)

def bs_price(S,K,r,q,T,sigma,option_type):
    if option_type == 'C':
        return np.exp(-q*T)*S*N(d1(S,K,r,q,T,sigma))-np.exp(-r*T)*K*N(d2(S,K,r,q,T,sigma))
    elif option_type == 'P':
        return -np.exp(-q*T)*S*N(-d1(S,K,r,q,T,sigma))+np.exp(-r*T)*K*N(-d2(S,K,r,q,T,sigma))
    
def bs_delta(S,K,r,q,T,sigma,option_type):
    if option_type == 'C':
        return np.exp(-q*T)*N(d1(S,K,r,q,T,sigma))
    elif option_type == 'P':
        return -np.exp(-q*T)*N(-d1(S,K,r,q,T,sigma))

def load_warrant_historical_prices(ticker):
    df = pd.read_excel('case_study_q1_data.xlsx', sheet_name='Warrant history', skiprows=3)
    
    df.rename(columns={
        'LVWR/WS': 'LVWR/WS_date',
        'Unnamed: 1': 'LVWR/WS_price',
        'TLSIW': 'TLSIW_date',
        'Unnamed: 3': 'TLSIW_price',
        'AFRIW': 'AFRIW_date',
        'Unnamed: 5': 'AFRIW_price',
        'CLBTW': 'CLBTW_date',
        'Unnamed: 7': 'CLBTW_price',
        'GRND/WS': 'GRND/WS_date',
        'Unnamed: 9': 'GRND/WS_price',
    }, inplace=True)
    
    df = df.loc[:,df.columns.str.startswith(ticker)]
    df.dropna(inplace=True)
    
    df.set_index(ticker+'_date', inplace=True)
    df.rename_axis(index={ticker+'_date': 'date'}, inplace=True)
    
    return df

def load_stock_historical_prices(ticker):
    df = pd.read_excel('case_study_q1_data.xlsx', sheet_name='Stock history', skiprows=2)
    
    df.rename(columns={
        'LVWR': 'LVWR_date',
        'LVWR.1': 'LVWR_price',
        'TLSI': 'TLSI_date',
        'TLSI.1': 'TLSI_price',
        'AFRI': 'AFRI_date',
        'AFRI.1': 'AFRI_price',
        'CLBT': 'CLBT_date',
        'CLBT.1': 'CLBT_price',
        'GRND': 'GRND_date',
        'GRND.1': 'GRND_price',
    }, inplace=True)
    
    df = df.loc[:,df.columns.str.startswith(ticker)]
    df.dropna(inplace=True)
    
    df.set_index(ticker+'_date', inplace=True)
    df.rename_axis(index={ticker+'_date': 'date'}, inplace=True)
    
    return df

def load_warrant_static_data(ticker):
    df = pd.read_excel('case_study_q1_data.xlsx', sheet_name='Warrant static data', skiprows=1)
    
    df.rename(columns={
        'Unnamed: 1': 'field'
    }, inplace=True)
    
    df.set_index('field', inplace=True)
    
    df = df.loc[:,df.columns.str.startswith(ticker)]    
    
    return df.T

def load_etf_historical_prices(ticker):
    df = pd.read_excel('case_study_q1_data.xlsx', sheet_name='ETFs', skiprows=2).iloc[:,:4]
    
    df.rename(columns={
        'IWM': 'IWM_date',
        'Unnamed: 1': 'IWM_price',
        'SPY': 'SPY_date',
        'Unnamed: 3': 'SPY_price',
    }, inplace=True)
    
    df = df.loc[:,df.columns.str.startswith(ticker)]
    df.dropna(inplace=True)
    
    df.set_index(ticker+'_date', inplace=True)
    df.rename_axis(index={ticker+'_date': 'date'}, inplace=True)
    
    return df

def load_etf_historical_dividends(ticker):
    df = pd.concat([
        pd.read_excel('case_study_q1_data.xlsx', sheet_name='ETFs', skiprows=3).iloc[:,5:12],
        pd.read_excel('case_study_q1_data.xlsx', sheet_name='ETFs', skiprows=3).iloc[:,13:20]],
        axis=0
    )
    
    df.rename(columns={
        'Declared date': 'IWM_declared_date',
        'Ex date': 'IWM_ex_date',
        'Record Date': 'IWM_record_date',
        'Pay date': 'IWM_pay_date',
        'Net amount': 'IWM_net_amount',
        'Frequency': 'IWM_frequency',
        'Type': 'IWM_type',
        'Declared date.1': 'SPY_declared_date',
        'Ex date.1': 'SPY_ex_date',
        'Record Date.1': 'SPY_record_date',
        'Pay date.1': 'SPY_pay_date',
        'Net amount.1': 'SPY_net_amount',
        'Frequency.1': 'SPY_frequency',
        'Type.1': 'SPY_type',

    }, inplace=True)
    
    df = df.loc[:,df.columns.str.startswith(ticker)]
    df.dropna(inplace=True)
    
    df.set_index(ticker+'_declared_date', inplace=True)
    
    return df

def calculate_theoretical_delta(u_ticker, w_ticker):
    
    df_und = load_stock_historical_prices(u_ticker)
    u_date = pd.to_datetime(df_und.iloc[[-1]].index.values[0]).date()
    u_price = df_und.iloc[-1,0]

    #estimate implied vol using realized volatility of stock
    df_und['return'] = np.log(df_und[u_ticker+'_price'] / df_und[u_ticker+'_price'].shift(1))
    iv = df_und['return'].dropna().std()*np.sqrt(252)

    df_wrt = load_warrant_static_data(w_ticker)
    expiry = pd.to_datetime(df_wrt['Expiry'], format='%m/%d/%Y', errors='coerce').iloc[0].date()
    strike = pd.to_numeric(df_wrt['Strike'], errors='coerce').iloc[0]
    years_to_expiry = (expiry - u_date).days / 365

    return bs_delta(S=u_price, K=strike, r=0.055, q=0, T=years_to_expiry, sigma=iv, option_type='C')

def calculate_delta_notional(eval_date, w_tkr, qty):
    df_wrt = load_warrant_static_data(w_tkr)
    expiry = pd.to_datetime(df_wrt['Expiry'], format='%m/%d/%Y', errors='coerce').iloc[0].date()
    strike = pd.to_numeric(df_wrt['Strike'], errors='coerce').iloc[0]
    years_to_expiry = (expiry - eval_date).days / 365

    d = bs_delta(S=u_price, K=strike, r=0.055, q=0, T=years_to_expiry, sigma=iv, option_type='C')    
    
    return qty * strike * d

def calculate_ex_post_delta(df_deriv, df_under):

    df = df_under.join(df_deriv, how='left')
    df.ffill(inplace=True)
    df.dropna(inplace=True)

    df[df_under.columns[0]+'_change'] = df[df_under.columns[0]] - df[df_under.columns[0]].shift(1)
    df[df_deriv.columns[0]+'_change'] = df[df_deriv.columns[0]] - df[df_deriv.columns[0]].shift(1)

    df.dropna(axis=0, how='any', inplace=True)
    df = df.loc[df[df_under.columns[0]+'_change'] != 0]
    df.fillna(0., inplace=True)

    df['change_ratio'] = df[df_deriv.columns[0]+'_change'] / df[df_under.columns[0]+'_change']

    return df['change_ratio'], df[df_deriv.columns[0]+'_change'], df[df_under.columns[0]+'_change']

def calculate_geometric_mean(prices):
    
    return prices.prod(axis=1)**(1/len(prices.columns))

#### 1) Quantify this security’s theoretical delta using a Black-Scholes model based on recent observable data (approximations are fine, no need to derive BS model here).

In [2]:
df_und = load_stock_historical_prices('LVWR')
u_date = pd.to_datetime(df_und.iloc[[-1]].index.values[0]).date()
u_price = df_und.iloc[-1,0]

#estimate implied vol using realized volatility of stock
df_und['return'] = np.log(df_und['LVWR_price'] / df_und['LVWR_price'].shift(1))
iv = df_und['return'].dropna().std()*np.sqrt(252)

df_wrt = load_warrant_static_data('LVWR')
expiry = pd.to_datetime(df_wrt['Expiry'], format='%m/%d/%Y', errors='coerce').iloc[0].date()
strike = pd.to_numeric(df_wrt['Strike'], errors='coerce').iloc[0]
years_to_expiry = (expiry - u_date).days / 365

ex_ante_delta = bs_delta(S=u_price, K=strike, r=0.055, q=0, T=years_to_expiry, sigma=iv, option_type='C')

print(f"Black-Scholes delta as of {u_date.strftime('%Y-%m-%d')}: {ex_ante_delta:0.2f}")

Black-Scholes delta as of 2024-04-30: 0.62


#### 2) Quantify this security’s observed (realized) delta over recent history

In [14]:
df_under = load_stock_historical_prices('LVWR')
df_deriv = load_warrant_historical_prices('LVWR/WS')

ratio, w, u = calculate_ex_post_delta(df_deriv, df_under)

print(f"average ex-post delta: {ratio.mean():0.2f}")
ratio.iplot(kind='bar', title='Daily ex-post delta for LVWR/WS and LVWR', xTitle='Price Date', yTitle='Ratio of price changes')

average ex-post delta: 0.09


#### 3) How do these measures compare?

In [4]:
print(f"realized delta is {ratio.mean()/ex_ante_delta:0.2%} of most recent theoretical delta")

realized delta is 13.80% of most recent theoretical delta


#### 4) What factors do you believe are affecting security pricing?

In [5]:
n_obs_wrt = load_warrant_historical_prices('LVWR/WS').shape[0]
n_obs_und = load_stock_historical_prices('LVWR').shape[0]

print(f"Warrant volume is likely very low relative to underlying shares.")
print(f"There were {n_obs_und - n_obs_wrt} days when the underlying shares traded and the warrants did not.")
print(f"We're comparing a recent Black-Scholes delta rather than re-computing each day of the period we are analyzing")

Warrant volume is likely very low relative to underlying shares.
There were 14 days when the underlying shares traded and the warrants did not.
We're comparing a recent Black-Scholes delta rather than re-computing each day of the period we are analyzing


#### 5) Quantify the theoretical delta 1% risk of this portfolio using real world inputs. Please detail any assumptions that you make

In [6]:
pd.options.display.float_format = "{:0,.2f}".format

positions = {
    'LVWR/WS': ['LVWR', 'LVWR/WS', 1e6],
    'TLSIW': ['TLSI', 'TLSIW', 1e6],
    'AFRIW': ['AFRI', 'AFRIW', 1e6],
    'CLBTW': ['CLBT', 'CLBTW', 1e6],
    'GRND/WS': ['GRND', 'GRND/WS', 1e6],
}

results = {}

for name, pos in positions.items():
    
    
    u_tkr = pos[0]
    w_tkr = pos[1]
    w_qty = pos[2]
    
    df_wrt = load_warrant_static_data(w_tkr)
    expiry = pd.to_datetime(df_wrt['Expiry'], format='%m/%d/%Y', errors='coerce').iloc[0].date()
    strike = pd.to_numeric(df_wrt['Strike'], errors='coerce').iloc[0]
    years_to_expiry = (expiry - u_date).days / 365
    
    df_under = load_stock_historical_prices(u_tkr)
    df_deriv = load_warrant_historical_prices(w_tkr)
    
    ratio, deriv, under = calculate_ex_post_delta(df_deriv, df_under)
    
    results[name] = [
        
        expiry,
        strike,
        calculate_theoretical_delta(u_tkr, w_tkr),
        ratio.mean(),
        calculate_delta_notional(date(2024,4,30), w_tkr, w_qty)
    ]
    
    

port = pd.DataFrame.from_dict(results, orient='index', columns=['expiry', 'strike', 'ex_ante_delta', 'ex_post_delta', 'exposure'])

display(port)

print(f"Portfolio delta 1%: {port[['exposure']].sum().iloc[0] * 0.01:0,.0f}")

,expiry,strike,ex_ante_delta,ex_post_delta,exposure
LVWR/WS,2027-09-27,11.50,0.62,0.09,"7,135,867.43"
TLSIW,2028-08-10,11.50,0.70,0.09,"7,732,268.69"
AFRIW,2027-06-09,11.50,0.64,-0.48,"6,890,762.29"
CLBTW,2026-08-30,11.50,0.66,0.02,"6,125,436.17"
GRND/WS,2027-11-18,11.50,0.66,0.17,"7,243,886.49"


Portfolio delta 1%: 351,282


#### 6) Would the answer change if you were asked for the delta 10% or delta 20%? If so, why?

In [7]:
print(f"The exposure of the warrants portfolio is positively convex with respect to increases in the underlying shares.")
print(f"As the underlying move is increased the difference between estimated and true delta will increase.")

The exposure of the warrants portfolio is positively convex with respect to increases in the underlying shares.
As the underlying move is increased the difference between estimated and true delta will increase.


#### 7) What has the observable (realized) $ delta of this portfolio been YTD?

In [8]:
pd.options.display.float_format = "{:0,.2f}".format

positions = {
    'LVWR/WS': ['LVWR', 'LVWR/WS', 1e6],
    'TLSIW': ['TLSI', 'TLSIW', 1e6],
    'AFRIW': ['AFRI', 'AFRIW', 1e6],
    'CLBTW': ['CLBT', 'CLBTW', 1e6],
    'GRND/WS': ['GRND', 'GRND/WS', 1e6],
}

results = {}

port_stocks = port_warrants = pd.DataFrame()

for name, pos in positions.items():
        
    u_tkr = pos[0]
    w_tkr = pos[1]
    w_qty = pos[2]
        
    df_wrt = load_warrant_static_data(w_tkr)    
    expiry = pd.to_datetime(df_wrt['Expiry'], format='%m/%d/%Y', errors='coerce').iloc[0].date()
    strike = pd.to_numeric(df_wrt['Strike'], errors='coerce').iloc[0]
    years_to_expiry = (expiry - u_date).days / 365
    
    df_wrt_hist = load_warrant_historical_prices(w_tkr)
    
    df_und_hist = load_stock_historical_prices(u_tkr)
    u_date = pd.to_datetime(df_und.iloc[[-1]].index.values[0]).date()
    u_price = df_und.iloc[-1,0]
    
    #estimate implied vol using realized volatility of stock
    df_und_hist['return'] = np.log(df_und_hist[u_tkr+'_price'] / df_und_hist[u_tkr+'_price'].shift(1))
    iv = df_und['return'].dropna().std()*np.sqrt(252)
    
    d = bs_delta(u_price, strike, r=0.055, q=0, T=years_to_expiry, sigma=iv, option_type='C')
    ratio, w, u = calculate_ex_post_delta(df_wrt_hist, df_und_hist)
    
    results[name] = [
        
        expiry,
        strike,
        d,        
        ratio.mean()
    ]
    
    port_stocks = pd.concat([port_stocks, load_stock_historical_prices(u_tkr)], axis=1)
    port_warrants = pd.concat([port_warrants, load_warrant_historical_prices(w_tkr)], axis=1)

port = pd.DataFrame.from_dict(results, orient='index', columns=['expiry', 'strike', 'ex_post_delta', 'ex_post_delta'])

port_stocks.fillna(method='ffill', inplace=True)
port_warrants.fillna(method='ffill', inplace=True)

port_stocks['port_stocks_price'] = calculate_geometric_mean(port_stocks)
port_warrants['port_warrants_price'] = calculate_geometric_mean(port_warrants)
port_warrants['port_warrants_return'] = np.log(port_warrants['port_warrants_price'] / port_warrants['port_warrants_price'].shift(1))
port_warrants.dropna(how='any', inplace=True)

ratio, w, u = calculate_ex_post_delta(port_warrants[['port_warrants_price']], port_stocks[['port_stocks_price']])

display(port)

print(f"portfolio ex-post delta: {ratio.mean():0.2f}")
ratio.iplot(kind='bar', title='Portfolio daily ex-post delta', xTitle='Price Date', yTitle='Ratio of price changes')


,expiry,strike,ex_post_delta,ex_post_delta
LVWR/WS,2027-09-27,11.50,0.62,0.09
TLSIW,2028-08-10,11.50,0.67,0.08
AFRIW,2027-06-09,11.50,0.60,-0.48
CLBTW,2026-08-30,11.50,0.53,0.02
GRND/WS,2027-11-18,11.50,0.63,0.17


portfolio ex-post delta: 0.04


#### 8) If individual stock shorts were not available to hedge this portfolio but the broad market ETFs SPY and IWM (your only 2 choices) were available, how would you hedge this risk and why? Please quantify your recommendations.

#### SPY as a hedge

In [9]:
etf_tkr = 'SPY'

df_divs = load_etf_historical_dividends(etf_tkr)
df_prices = load_etf_historical_prices(etf_tkr)
df_prices = df_prices.join(df_divs[[etf_tkr+'_net_amount']])
df_prices = df_prices.bfill().fillna(0.)
df_prices[etf_tkr+'_price_adjusted'] = df_prices[etf_tkr+'_price'] - df_prices[etf_tkr+'_net_amount']
df_prices[etf_tkr+'_return_adjusted'] = np.log(df_prices[etf_tkr+'_price_adjusted'] / df_prices[etf_tkr+'_price_adjusted'].shift(1))
df_prices[etf_tkr+'_return_raw'] = np.log(df_prices[etf_tkr+'_price'] / df_prices[etf_tkr+'_price'].shift(1))
df_prices.dropna(inplace=True)
coef = np.polyfit(df_prices[etf_tkr+'_return_adjusted'], port_warrants['port_warrants_return'], 1)

df_plot = pd.concat([df_prices[etf_tkr+'_return_adjusted'], port_warrants['port_warrants_return']], axis=1)

spy_corr = df_plot.corr().iloc[0,1]
print(f"Use a hedge ratio of {-coef[0]:0.2f} when hedging the warrants portfolio with {etf_tkr}")

Use a hedge ratio of -0.39 when hedging the warrants portfolio with SPY


In [10]:
df_plot.iplot(
    kind='scatter', 
    x=etf_tkr+'_return_adjusted', 
    y='port_warrants_return', 
    mode='markers', 
    title='daily returns', 
    xTitle=etf_tkr,
    xaxis_tickformat='0.2%',
    yTitle='Warrants Portfolio',
    yaxis_tickformat='0.2%',
    bestfit=True
)

#### IWM as a hedge

In [11]:
etf_tkr = 'IWM'

df_divs = load_etf_historical_dividends(etf_tkr)
df_prices = load_etf_historical_prices(etf_tkr)
df_prices = df_prices.join(df_divs[[etf_tkr+'_net_amount']])
df_prices = df_prices.bfill().fillna(0.)
df_prices[etf_tkr+'_price_adjusted'] = df_prices[etf_tkr+'_price'] - df_prices[etf_tkr+'_net_amount']
df_prices[etf_tkr+'_return_adjusted'] = np.log(df_prices[etf_tkr+'_price_adjusted'] / df_prices[etf_tkr+'_price_adjusted'].shift(1))
df_prices[etf_tkr+'_return_raw'] = np.log(df_prices[etf_tkr+'_price'] / df_prices[etf_tkr+'_price'].shift(1))
df_prices.dropna(inplace=True)
coef = np.polyfit(df_prices[etf_tkr+'_return_adjusted'], port_warrants['port_warrants_return'], 1)

df_plot = pd.concat([df_prices[etf_tkr+'_return_adjusted'], port_warrants['port_warrants_return']], axis=1)
iwm_corr = df_plot.corr().iloc[0,1]

print(f"Use a hedge ratio of {-coef[0]:0.2f} when hedging the warrants portfolio with {etf_tkr}")

Use a hedge ratio of -0.32 when hedging the warrants portfolio with IWM


In [12]:
df_plot.iplot(
    kind='scatter', 
    x=etf_tkr+'_return_adjusted', 
    y='port_warrants_return', 
    mode='markers', 
    title='daily returns', 
    xTitle=etf_tkr,
    xaxis_tickformat='0.2%',
    yTitle='Warrants Portfolio',
    yaxis_tickformat='0.2%',
    bestfit=True
)

#### 9) What additional risks does hedging in this manner introduce?

In [13]:
print(f"The market etfs only explain a tiny amount of the warrants portfolio risk and introduce additional risk factors")
print(f"SPY explains {spy_corr**2:0.2%} of the warrant portfolio's price movements")
print(f"IWM explains {iwm_corr**2:0.2%} of the warrant portfolio's price movements")

The market etfs only explain a tiny amount of the warrants portfolio risk and introduce additional risk factors
SPY explains 1.32% of the warrant portfolio's price movements
IWM explains 2.58% of the warrant portfolio's price movements
